In [61]:
import nltk
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mohammad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mohammad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
df_total = pd.read_csv('train_test.csv')
df_train, df_test = train_test_split(df_total, test_size=0.2, shuffle=True)

,type,text
1880,ham,"If i let you do this, i want you in the house ..."
3688,ham,"Well the general price is &lt;#&gt; /oz, let ..."
5032,ham,happened here while you were adventuring
4058,ham,Think + da. You wil do.
4481,ham,Slept? I thinkThis time ( &lt;#&gt; pm) is no...
...,...,...
2384,ham,ÌÏ still attending da talks?
744,ham,Now? I'm going out 4 dinner soon..
3113,ham,Ok c Ì_ then.
2098,ham,Okie... Thanx...


In [48]:
len(stopwords.words('english'))

179

In [68]:
def preprocess_string(s):
    s = s.lower()                       # only use lowercase characters
    s = ' '.join(s.split())             # remove extra whitespace
    tokens = word_tokenize(s)           # tokenization
    
    # remove stopwords
    swords = stopwords.words('english')
    nonstop_tokens = []
    for token in tokens:
        if token not in swords:
            nonstop_tokens.append(token)

    # remove punctuations
    regexp_tokenizer = RegexpTokenizer(r'\w+')
    nonstop_nonpunc_tokens = regexp_tokenizer.tokenize(' '.join(nonstop_tokens))

    s = ' '.join(nonstop_nonpunc_tokens)
    return s


In [69]:
old_string = df_train.sample().iloc[0]['text']
new_string = preprocess_string(old_string)
print('Sample text:', old_string)
print('Normalized text:', new_string)

Sample text: URGENT, IMPORTANT INFORMATION FOR O2 USER. TODAY IS YOUR LUCKY DAY! 2 FIND OUT WHY LOG ONTO HTTP://WWW.URAWINNER.COM THERE IS A FANTASTIC SURPRISE AWAITING FOR YOU
Normalized text: urgent important information o2 user today lucky day 2 find log onto http www urawinner com fantastic surprise awaiting


In [14]:
def preprocess(df):
    new_df = df.copy()
    for idx, row in new_df.iterrows():
        print(rpw['type'], row['text'])

preprocess(df_train)

ham If i let you do this, i want you in the house by 8am.
ham Well the general price is  &lt;#&gt; /oz, let me know if/when/how much you want
ham happened here while you were adventuring
ham Think + da. You wil do.
ham Slept? I thinkThis time ( &lt;#&gt;  pm) is not dangerous
ham Did u fix the teeth?if not do it asap.ok take care.
ham Desires- u going to doctor 4 liver. And get a bit stylish. Get ur hair managed. Thats it.
ham Just getting back home
spam CALL 09090900040 & LISTEN TO EXTREME DIRTY LIVE CHAT GOING ON IN THE OFFICE RIGHT NOW TOTAL PRIVACY NO ONE KNOWS YOUR [sic] LISTENING 60P MIN 24/7MP 0870753331018+
ham He remains a bro amongst bros
ham K k:) sms chat with me.
ham I want to tel u one thing u should not mistake me k THIS IS THE MESSAGE THAT YOU SENT:)
ham Short But Cute: \Be a good person
ham Thats cool. How was your day?
ham No drama Pls.i have had enough from you and family while i am struggling in the hot sun in a strange place.No reason why there should be an ego of 